# 第51回衆議院議員総選挙 YouTube データ分析
## 2026年2月8日投開票

このノートブックでは、YouTube Data API v3 から収集したデータを用いて、
選挙関連動画のトレンド・争点分析・政党比較・世論分析を行います。

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

try:
    import japanize_matplotlib
except ImportError:
    plt.rcParams['font.family'] = 'Hiragino Sans'

sns.set_theme(style='whitegrid', font_scale=1.2)

# サンプルデータがない場合は生成
from pathlib import Path
if not (Path('../data/processed/daily_video_counts.csv')).exists():
    from generate_sample_data import generate_all_sample_data
    generate_all_sample_data()

print('準備完了!')

## 1. データの読み込み

In [ ]:
daily_counts = pd.read_csv('../data/processed/daily_video_counts.csv')
daily_views = pd.read_csv('../data/processed/daily_views.csv')
issue_stats = pd.read_csv('../data/processed/issue_stats.csv')
channel_stats = pd.read_csv('../data/processed/channel_analysis.csv')
party_stats = pd.read_csv('../data/processed/party_video_stats.csv')
sentiment = pd.read_csv('../data/processed/sentiment_counts.csv')

print(f'日別データ: {len(daily_counts)} 日分')
print(f'争点カテゴリ: {len(issue_stats)} 件')
print(f'政党チャンネル: {len(channel_stats)} 件')

## 2. 選挙関連動画の投稿数推移

公示日（1/27）以降、動画投稿数がどう変化したかを確認します。

In [ ]:
daily_counts['date'] = pd.to_datetime(daily_counts['date'])

fig = px.bar(daily_counts, x='date', y='video_count',
             title='選挙関連YouTube動画 日別投稿数推移',
             labels={'date': '日付', 'video_count': '動画数'})

# 公示日と投票日のライン
fig.add_vline(x=pd.Timestamp('2026-01-27').timestamp() * 1000,
              line_dash='dash', line_color='red',
              annotation_text='公示日 1/27')
fig.add_vline(x=pd.Timestamp('2026-02-08').timestamp() * 1000,
              line_dash='dash', line_color='blue',
              annotation_text='投票日 2/8')
fig.show()

## 3. 争点別の注目度比較

消費税・物価高、安全保障、移民政策など、各争点がYouTubeでどれだけ注目されているか比較します。

In [ ]:
fig = px.bar(issue_stats, x='total_views', y='issue', orientation='h',
             title='争点別 総再生回数',
             labels={'total_views': '総再生回数', 'issue': '争点'},
             color='issue',
             color_discrete_sequence=px.colors.qualitative.Set2)
fig.update_layout(showlegend=False, yaxis={'categoryorder': 'total ascending'})
fig.show()

In [ ]:
# 争点別 動画数 vs 平均再生回数 の散布図
fig = px.scatter(issue_stats, x='video_count', y='avg_views',
                 size='total_views', color='issue',
                 title='争点別: 動画数 vs 平均再生回数',
                 labels={'video_count': '動画数', 'avg_views': '平均再生回数'},
                 hover_name='issue')
fig.show()

## 4. 政党公式チャンネル比較

各政党の公式YouTubeチャンネルの登録者数・動画数・再生回数を比較します。

In [ ]:
party_colors = {
    '自由民主党': '#E3242B', '日本維新の会': '#3CB371',
    '立憲民主党': '#1E90FF', '国民民主党': '#FF8C00',
    '日本共産党': '#DC143C', 'れいわ新選組': '#FF69B4',
    '参政党': '#DAA520'
}

cs = channel_stats.dropna(subset=['party_name']).sort_values('subscriber_count')

fig = px.bar(cs, x='subscriber_count', y='party_name', orientation='h',
             title='政党別 YouTubeチャンネル登録者数',
             labels={'subscriber_count': '登録者数', 'party_name': '政党'},
             color='party_name',
             color_discrete_map=party_colors)
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
# 政党チャンネル 総再生回数
fig = px.bar(cs.sort_values('view_count'), x='view_count', y='party_name',
             orientation='h',
             title='政党別 YouTube総再生回数',
             labels={'view_count': '総再生回数', 'party_name': '政党'},
             color='party_name',
             color_discrete_map=party_colors)
fig.update_layout(showlegend=False)
fig.show()

## 5. コメント感情分析

選挙関連動画のコメントをポジティブ・ネガティブ・ニュートラルに分類した結果です。

In [ ]:
colors_map = {'positive': '#2ECC71', 'neutral': '#95A5A6', 'negative': '#E74C3C'}

fig = px.pie(sentiment, values='count', names='sentiment',
             title='選挙関連動画コメントの感情分布',
             color='sentiment',
             color_discrete_map=colors_map)
fig.show()

## 6. 選挙期間中の政党別動画パフォーマンス

In [ ]:
fig = px.bar(party_stats.sort_values('total_views', ascending=False),
             x='party_name', y='total_views',
             title='選挙期間中 政党別動画の総再生回数',
             labels={'party_name': '政党', 'total_views': '総再生回数'},
             color='party_name',
             color_discrete_map=party_colors)
fig.update_layout(showlegend=False)
fig.show()

## まとめ

### 分析から得られる知見
1. **投稿トレンド**: 公示日（1/27）を境に選挙関連動画の投稿が急増
2. **争点の注目度**: 消費税・物価高が最も関心を集めている
3. **政党のYouTube活用**: 各政党で戦略に差がある
4. **世論の傾向**: コメントの感情分析から有権者の反応を把握

### 注意事項
- YouTubeのデータは全有権者を代表するものではない
- アルゴリズムによる偏りがある可能性
- ボットによるコメント・再生の影響も考慮が必要